# Install detectron2

In [0]:
# install dependencies:
# (use +cu100 because colab is on CUDA 10.0)
# ----------------------UNCOMMENT FROM HERE----------------------------------
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# ---------------------UNCOMMENT TILL HERE-----------------------------------
# opencv is pre-installed on colab

In [0]:
# UNCOMMENT WHEN RUNNING THIS NOTEBOOK FOR THE FIRST TIME
!git clone -qq https://github.com/facebookresearch/detectron2 detectron2_repo

# You can replace cu101 with "cu{100,92}" or "cpu".
!pip install -qq detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
!cp detectron2_repo/configs ./ -r

Restart runtime in case you are installing detectron for the first time

In [0]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# common libraries
import numpy as np
import cv2
import random
import pandas as pd
import os

try:
    from google.colab.patches import cv2_imshow
except:
    os.system(f"""pip install google.colab""")
    from google.colab.patches import cv2_imshow
import glob
import matplotlib.pyplot as plt
# import some common .detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode

# Train on a custom dataset

If you want to use a custom dataset while also reusing detectron2’s data loaders, you will need to

* Register your dataset (i.e., tell detectron2 how to obtain your dataset).
* Optionally, register metadata for your dataset.

## Prepare the dataset

To register your dataset to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).
Here, the dataset is in its custom format, therefore we write a function to parse it and prepare it into detectron2's standard format. See the tutorial for more details.


In [0]:
!git clone https://github.com/Arnav0400/EfficientDet.Pytorch.git
!wget https://raw.githubusercontent.com/Arnav0400/EfficientDet.Pytorch/working-bbox/test.csv
!wget https://raw.githubusercontent.com/Arnav0400/EfficientDet.Pytorch/working-bbox/train.csv

In [0]:
train = pd.read_csv('train.csv')
val = pd.read_csv('test.csv')

In [0]:
train.label = 0
val.label = 0

In [0]:
val.head()

In [0]:
ls  

In [0]:
plt.imread('EfficientDet.Pytorch/boostnet_labeldata/data/training/sunhl-1th-01-Mar-2017-310 C AP.jpg')

In [0]:
for i tqdm(range(len(val))):
    plt.imread()

In [0]:
def get_bb(img_path, img_name, df):
    """
        This function returns annotations in the format
        xyxy_ABS: xmin, ymin, xmax, ymax - ABSOLUTE
        
        Function built taking into consideration the dataset
        provided by EAD2020
        
        change the path to the bounding box folder(if not training on EAD)
        and change the loading method(if using .mat) accordingly
    """
    # change the path to the bounding box folder(if not training on EAD)
    # and change the loading method accordingly
    bbox = df.loc[df.image_id == img_name].iloc[:, 1:5]
    label = df.label[df.image_id == img_name]
    return np.asarray(bbox), np.asarray(label)

def _get_dicts(phase):
    if phase == 'train':
        path = 'EfficientDet.Pytorch/boostnet_labeldata/data/training/'
        df = train
    elif phase == 'val':
        path = 'EfficientDet.Pytorch/boostnet_labeldata/data/test/'
        df = val
    else:
        raise(Exception('Provide either "Train" or "Val"'))
    
    def get_dicts():
        dataset_dicts = []
        img_list = os.listdir(path)
        for idx, i in enumerate(img_list):
            record = {}
            img = plt.imread(path+i)
            height, width = img.shape        
            record["file_name"] = path+i
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            proposal_bb, proposal_logits = get_bb(path, i, df)
            objs=[]
            for j in range(len(proposal_bb)):
                obj = {
                    "bbox": [proposal_bb[j][0], proposal_bb[j][1], proposal_bb[j][2], proposal_bb[j][3]],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "category_id": proposal_logits[j],
                    "iscrowd": 0
                }
                objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
        return dataset_dicts
    return get_dicts

from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]:
    DatasetCatalog.register("spine1_" + d, _get_dicts(d))
    if d == "train":
      MetadataCatalog.get("spine1_" + d).set(thing_classes=["Vertebrae"])
    elif d == "val":
      MetadataCatalog.get("spine1_" + d).set(thing_classes=["Vertebrae"],
                                          pred_classes=["Vertebrae"])
spine_metadata = MetadataCatalog.get("spine1_train") 
spine_metadata_val = MetadataCatalog.get("spine1_val") 

## Visualisation



In [0]:
spine_metadata.get("thing_classes", None)

In [0]:
# Uncomment if you want to visualize training data
dataset_dicts = _get_dicts('train')()
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=spine_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])
    

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### To choose model:
    detectron2_repo-->configs
    Accordingly choose Detection or whatever you require
    For eg:
        "detectron2_repo/configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml"
### To get weights:
    detectron2_repo-->detectron2-->model_zoo-->model_zoo.py
    For eg: copy any link
        "COCO-Detection/faster_rcnn_R_50_C4_3x.yaml": "137849393/model_final_f97cb7.pkl"
    delete the mid part and add detectron2:// to the start
        "detectron2://COCO-Detection/faster_rcnn_R_50_C4_3x/137849393/model_final_f97cb7.pkl"
    use this format as a link to get the model weights

# Train

In [0]:
from detectron2_repo.detectron2.data import transforms as T
import copy
import torch
from detectron2_repo.detectron2.data import detection_utils as utils
import os
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, verify_results
from detectron2.utils.logger import setup_logger

In [0]:
class Mapper:
    """
    A callable which takes a dataset dict in Detectron2 Dataset format,
    and map it into a format used by the model.
    This is the default callable to be used to map your dataset dict into training data.
    You may need to follow it to implement your own one for customized logic,
    such as a different way to read or transform images.
    See :doc:`/tutorials/data_loading` for details.
    The callable currently does the following:
    1. Read the image from "file_name"
    2. Applies cropping/geometric transforms to the image and annotations
    3. Prepare data and annotations to Tensor and :class:`Instances`
    """

    def __init__(self, cfg, is_train=True):
        if cfg.INPUT.CROP.ENABLED and is_train:
            self.crop_gen = T.RandomCrop(cfg.INPUT.CROP.TYPE, cfg.INPUT.CROP.SIZE)
            logging.getLogger(__name__).info("CropGen used in training: " + str(self.crop_gen))
        else:
            self.crop_gen = None

        # self.tfm_gens = utils.build_transform_gen(cfg, is_train)

        # fmt: off
        self.tfm_gens = utils.build_transform_gen(cfg, is_train)
        self.img_format     = cfg.INPUT.FORMAT
        self.mask_on        = cfg.MODEL.MASK_ON
        self.mask_format    = cfg.INPUT.MASK_FORMAT
        self.keypoint_on    = cfg.MODEL.KEYPOINT_ON
        self.load_proposals = cfg.MODEL.LOAD_PROPOSALS
        self.transform_list_mod = [T.RandomBrightness(0.8, 1.2),
                                  T.RandomContrast(0.8, 1.2),
                                  T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                                  T.RandomSaturation(0.8, 1.2)]
        # fmt: on
        if self.keypoint_on and is_train:
            # Flip only makes sense in training
            self.keypoint_hflip_indices = utils.create_keypoint_hflip_indices(cfg.DATASETS.TRAIN)
        else:
            self.keypoint_hflip_indices = None

        if self.load_proposals:
            self.min_box_side_len = cfg.MODEL.PROPOSAL_GENERATOR.MIN_SIZE
            self.proposal_topk = (
                cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN
                if is_train
                else cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TEST
            )
        self.is_train = is_train

    def __call__(self, dataset_dict):
        """
        Args:
            dataset_dict (dict): Metadata of one image, in Detectron2 Dataset format.
        Returns:
            dict: a format that builtin models in detectron2 accept
        """
        dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
        # USER: Write your own image loading if it's not from a file
        image = utils.read_image(dataset_dict["file_name"], format=self.img_format)
        utils.check_image_size(dataset_dict, image)

        if "annotations" not in dataset_dict:
            image, transforms = T.apply_transform_gens(
                self.transform_list_mod , image
            )
        else:
            # Crop around an instance if there are instances in the image.
            # USER: Remove if you don't use cropping
            if self.crop_gen:
                crop_tfm = utils.gen_crop_transform_with_instance(
                    self.crop_gen.get_crop_size(image.shape[:2]),
                    image.shape[:2],
                    np.random.choice(dataset_dict["annotations"]),
                )
                image = crop_tfm.apply_image(image)
            image, transforms = T.apply_transform_gens(self.tfm_gens, image)
            if self.crop_gen:
                transforms = crop_tfm + transforms

        image_shape = image.shape[:2]  # h, w

        # Pytorch's dataloader is efficient on torch.Tensor due to shared-memory,
        # but not efficient on large generic data structures due to the use of pickle & mp.Queue.
        # Therefore it's important to use torch.Tensor.
        dataset_dict["image"] = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)))

        # USER: Remove if you don't use pre-computed proposals.
        if self.load_proposals:
            utils.transform_proposals(
                dataset_dict, image_shape, transforms, self.min_box_side_len, self.proposal_topk
            )

        if not self.is_train:
            # USER: Modify this if you want to keep them for some reason.
            dataset_dict.pop("annotations", None)
            dataset_dict.pop("sem_seg_file_name", None)
            return dataset_dict

        if "annotations" in dataset_dict:
            # USER: Modify this if you want to keep them for some reason.
            for anno in dataset_dict["annotations"]:
                if not self.mask_on:
                    anno.pop("segmentation", None)
                if not self.keypoint_on:
                    anno.pop("keypoints", None)

            # USER: Implement additional transformations if you have other types of data
            annos = [
                utils.transform_instance_annotations(
                    obj, transforms, image_shape, keypoint_hflip_indices=self.keypoint_hflip_indices
                )
                for obj in dataset_dict.pop("annotations")
                if obj.get("iscrowd", 0) == 0
            ]
            instances = utils.annotations_to_instances(
                annos, image_shape, mask_format=self.mask_format
            )
            # Create a tight bounding box from masks, useful when image is cropped
            if self.crop_gen and instances.has("gt_masks"):
                instances.gt_boxes = instances.gt_masks.get_bounding_boxes()
            dataset_dict["instances"] = utils.filter_empty_instances(instances)

        # USER: Remove if you don't do semantic/panoptic segmentation.
        if "sem_seg_file_name" in dataset_dict:
            with PathManager.open(dataset_dict.pop("sem_seg_file_name"), "rb") as f:
                sem_seg_gt = Image.open(f)
                sem_seg_gt = np.asarray(sem_seg_gt, dtype="uint8")
            sem_seg_gt = transforms.apply_segmentation(sem_seg_gt)
            sem_seg_gt = torch.as_tensor(sem_seg_gt.astype("long"))
            dataset_dict["sem_seg"] = sem_seg_gt
        return dataset_dict

In [0]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluators = [COCOEvaluator(dataset_name, cfg, True, output_folder)]
        if cfg.MODEL.DENSEPOSE_ON:
            evaluators.append(DensePoseCOCOEvaluator(dataset_name, True, output_folder))
        return DatasetEvaluators(evaluators)

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name, mapper=Mapper(cfg, False))

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=Mapper(cfg, True))

In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()

config_file = "COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"
# config_file = "COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml"
# config_file = "COCO-Detection/retinanet_R_50_FPN_3x.yaml"
# config_file = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"

cfg.merge_from_file(f"configs/{config_file}")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_file)
# cfg.MODEL.WEIGHTS = f"detectron2://{config_file[:-5]}/{weights_url}"  # initialize from model zoo


cfg.DATASETS.TRAIN = ("spine1_train",)
cfg.DATASETS.TEST = ("spine1_train",)   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.RETINANET.NUM_CLASSES = 8

In [0]:
# Automatically switch to a new tensorboard directory every time this cell is executed
i = 1
name_desc = config_file  # Extended Name for the Run
notes= config_file + ": Serious run"  # Notes, if any

while os.path.exists(os.path.join(cfg.OUTPUT_DIR, f'run{i}')):
    i += 1

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 1500
cfg.OUTPUT_DIR = os.path.join('./output/', f'run{i}')
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Create trainer
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)

# WandB
config = dict(cfg)
del config['MODEL']

# Commence Training
# trainer.train()

## Inference & evaluation using the trained model



Then, we randomly select several samples to visualize the prediction results.

In [0]:
cfg.confidence_threshold = 0.4

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

cfg.MODEL.WEIGHTS = os.path.join("drive/My Drive/Model_RCNN/model_final.pth")

trainer.resume_or_load(resume=False)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = cfg.confidence_threshold 

In [0]:
!wget --header="Host: n1cmcifi.gradient.paperspace.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://n1cmcifi.gradient.paperspace.com/tree/storage" --header="Cookie: _gcl_au=1.1.1667131129.1580132314; _ga=GA1.2.2106986882.1580132314; ajs_group_id=null; hubspotutk=3a07ae068ddf7242eec0cf8cf7ec51f3; __hssrc=1; _fbp=fb.1.1580132326512.1575742389; __cfduid=de4cc529f615defbf1d33c67f101a56641582793600; ajs_user_id=%22725331%22; ajs_anonymous_id=%22e050b6e1-31ec-481c-981c-eb5f56f20dc7%22; username-n1cmcifi-gradient-paperspace-com="2|1:0|10:1583573211|41:username-n1cmcifi-gradient-paperspace-com|44:ODM4NWFmOTk0ODA1NDIwM2FhMWVkNzFlZDVmZjlkY2Y=|979e0a6f949fc7bad5f24e73f6f4a50a08c185cb717f9a0d97fb549a4f90be54"; _xsrf=2|a7c16ff7|cd25302fb971ff9512e6b5604a366fd3|1583573211; _gid=GA1.2.1664767584.1583591959; _hp2_ses_props.2513992153=%7B%22ts%22%3A1583591958273%2C%22d%22%3A%22www.paperspace.com%22%2C%22h%22%3A%22%2Faccount%2Flogin%22%2C%22q%22%3A%22%3Fredirect%3D%252Fconsole%22%7D; __hstc=56075956.3a07ae068ddf7242eec0cf8cf7ec51f3.1580132318038.1583420339666.1583591959669.31; _cio=5dbed8eb-73b3-82dd-92e4-2a2457f72fee; _hp2_id.2513992153=%7B%22userId%22%3A%227208498489345016%22%2C%22pageviewId%22%3A%221861414297015364%22%2C%22sessionId%22%3A%221704883014752701%22%2C%22identity%22%3A%22725331%22%2C%22trackerVersion%22%3A%224.0%22%2C%22identityField%22%3Anull%2C%22isIdentified%22%3A1%2C%22oldIdentity%22%3Anull%7D; __hssc=56075956.4.1583591959669" --header="Connection: keep-alive" "https://n1cmcifi.gradient.paperspace.com/files/storage/d4-resize-(1536%2C512)c.pth?download=1" -O "d4-resize-(1536,512)c.pth" -c

In [0]:
# evaluator = COCOEvaluator("endo1_val", cfg, False, output_dir="./output/")
# val_loader = build_detection_test_loader(cfg, "endo1_val")
# inference_on_dataset(trainer.model, val_loader, (evaluator, ))
# trainer.test(, cfg, )
# evaluator = COCOEvaluator("endo1_val", cfg, False, output_dir="./output/")
# val_loader = build_detection_test_loader(cfg, "endo1_val")
# inference_on_dataset(trainer.model, val_loader, evaluator)

In [0]:
??Visualizer

In [0]:
from detectron2.utils.visualizer import ColorMode

cfg.DATASETS.TEST = ("spine1_val", )
predictor = DefaultPredictor(cfg)

for t in ['train', 'val']:
    dataset_dicts = _get_dicts(t)()

    print("\n"*2 + t.upper() + "\n"*2)
    for d in random.sample(dataset_dicts, 10):
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=MetadataCatalog.get(f"spine1_{t}"), 
                       scale=0.8, 
#                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#         v = v.draw_instance_predictions(outputs["proposals"].to("cpu"))
        cv2_imshow(v.get_image()[:, :, ::-1])
        os.makedirs(os.path.join(cfg.OUTPUT_DIR, f'{t}_preds/'), exist_ok=True)
        
        # WandB
        # wandb.log({f"{t}: {d['file_name']}": [wandb.Image(v.get_image(), caption=f"Threshold: {cfg.confidence_threshold}")]})

# Predict 

In [0]:
def _get_dicts_test(folder='EfficientDet.Pytorch/boostnet_labeldata/data/test'):
    path = folder
    def get_dicts_test():
        dataset_dicts = []
        img_list = os.listdir(path)
        for idx, i in enumerate(img_list):
            record = {}
            try:
                # print(os.path.join(path, i))
                img = cv2.imread(os.path.join(path, i))
                height, width, _ = img.shape
            except Exception as e:
                print(e)
                continue
            record["file_name"] = os.path.join(path, i)
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            record["thing_classes"] = ["Vertebrae"]
            dataset_dicts.append(record)
        return dataset_dicts
    return get_dicts_test


def create_txt(a, name):
    """
       This function will create a .txt file of the name given
       or append to the existing one.
    """

    save_path = 'val_preds'
    os.makedirs(save_path, exist_ok=True)
    
    path=f"{save_path}/{name}"
    
    pred_boxes = a['pred_boxes'].tensor.detach().cpu().numpy()
    scores = a['scores'].detach().cpu().numpy()
    pred_classes = a['pred_classes'].detach().cpu().numpy()
    names = {0:'Vertebrae'}
    
    f = open(path, 'a+')
    for i in range(len(pred_boxes)):
        x1, y1, x2, y2 = pred_boxes[i]
        label = pred_classes[i]
        confidence = scores[i]
        content = "Vertebrae"+" "+str(confidence)+" "+str(int((x1)))+" "+str(int(y1))+" "+str(int(x2))+" "+str(int(y2))+"\n"
        f.write(content)
    print(f'\rSaved {path}', end='')

In [0]:
!git clone 'https://github.com/rafaelpadilla/Object-Detection-Metrics.git'

In [0]:
from tqdm.notebook import tqdm

In [0]:
outputs[1]['instances'].get_fields()

In [0]:
cfg.confidence_threshold = 0.5
cfg.MODEL.WEIGHTS = os.path.join("drive/My Drive/Model_RCNN/model_final.pth")
visualize_preds = True
trainer.resume_or_load(resume=False)


cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = cfg.confidence_threshold 

j = 1
# while os.path.exists(f"spine_test{j}"):
#     j += 1
#     j -= 1
# os.makedirs(f"spine_test{j}")
folder = 'test'
# for folder in ['Detection', 'Detection_sequence', 'Generalization']:
print(f'\n\n{folder}\n\n')
folder = os.path.join('EfficientDet.Pytorch/boostnet_labeldata/data/', folder)
dataset_dicts = _get_dicts_test(folder=folder)()
outputs = []
files = []
print('reached checkpoint 1')
predictor = DefaultPredictor(cfg)
print('reached checkpoint 2')
print(f'Predicting on {folder}...', end='')
for d in dataset_dicts:
    im = cv2.imread(d["file_name"])
    output = predictor(im)
    outputs.append(output)
    files.append(f"{os.path.basename(d['file_name'])[:-4]}.txt")
print('Done!')

if visualize_preds:
    print('Sample Predictions')
    for i in random.sample(range(len(outputs)), 3):
        output, file = outputs[i], files[i]
        im = cv2.imread(os.path.join(folder, file[:-4] + '.jpg'))
        v = Visualizer(im[:, :, ::-1],
                    metadata=MetadataCatalog.get(f"spine1_val"), 
                    scale=0.8, 
        )
        v = v.draw_instance_predictions(output["instances"].to("cpu"))
        cv2_imshow(v.get_image()[:, :, ::-1])

    for i in tqdm(range(len(outputs))):
        output = outputs[i]['instances'].get_fields()
        create_txt(output, files[i])

## Try on any folder

#### Pass the csv with bounding box info and get a folder created with .txt files of the same name

In [0]:
def create_txt(boxes):
    """
       This function will create a .txt file of the name given
       or append to the existing one.
    """

    save_path = 'ground_truth'
    os.makedirs(save_path, exist_ok=True)
    
    for i in range(len(boxes)):
        name = boxes.image_id.loc[i][:-3]+'txt'
        path=f"{save_path}/{name}"
        f = open(path, 'a+')
        x1, y1, x2, y2 = np.asarray(boxes.iloc[i, 1:5])
        label = 1
        content = "Vertebrae"+" "+str(int((x1)))+" "+str(int(y1))+" "+str(int(x2))+" "+str(int(y2))+"\n"
        f.write(content)
    print(f'\rSaved {path}', end='')

In [0]:
# !cp -r 'drive/My Drive/val_preds' '.'

In [0]:
# !rm -r ground_truth

In [0]:
boxes_train = pd.read_csv('train.csv')
boxes_val = pd.read_csv('test.csv')

In [0]:
create_txt(boxes_val)

In [0]:
import sys
sys.path.insert(0, 'Object_Detection_Metrics/lib/')

In [0]:
from Object_Detection_Metrics.lib import BoundingBox
from Object_Detection_Metrics.lib import Evaluator

In [0]:
evalu= Evaluator.Evaluator()

In [0]:
val_txts = sorted(glob.glob('val_preds/*'))
gt_txts = sorted(glob.glob('ground_truth/*'))

In [0]:
def get_bboxes(val_txt, gt_txt):
  f_val = open(val_txt,'r')
  f_gt = open(gt_txt,'r')
  bbs_val=[]
  bbs_gt = []
  while(True):
    val_bb = f_val.readline()
    gt_bb = f_gt.readline()
    print(val_bb)
    img_name = 'a'
    classId = '1'
    try:
      conf,x,y,x2,y2 = float(val_bb.split(' ')[1]),int(val_bb.split(' ')[2]), int(val_bb.split(' ')[3]), int(val_bb.split(' ')[4]), int(val_bb.split(' ')[5])
      x_gt,y_gt,x2_gt,y2_gt = int(gt_bb.split(' ')[1]), int(gt_bb.split(' ')[2]), int(gt_bb.split(' ')[3]), int(gt_bb.split(' ')[4])
      print(y2)
      bbs_val.append(BoundingBox.BoundingBox(imageName=img_name,classId = classId, x=x,y=y,w=x2-x,h=y2-y,imgSize=None,bbType=BBType.Detected,classConfidence=conf,
                  format=BBFormat.XYWH))
      bbs_gt.append(BoundingBox.BoundingBox(imageName=img_name,classId = classId, x=x_gt,y=y_gt,w=x2_gt-x_gt,h=y2_gt-y_gt,imgSize=None,bbType=BBType.GroundTruth,classConfidence=None,
                  format=BBFormat.XYWH))
    except:
      break
  return [bbs_val,bbs_gt]

In [0]:
ious = []
for val_txt,gt_txt in zip(val_txts, gt_txts):
  [bbs_val,bbs_gt] = get_bboxes(val_txt,gt_txt)
  # print(bbs_val,bbs_gt)
  # print(len(bbs_val))
  for bb_val in bbs_val:
    iou_max = 0
    for bb_gt in bbs_gt:
      iou= evalu.iou(bb_val.getAbsoluteBoundingBox(BBFormat.XYX2Y2),bb_gt.getAbsoluteBoundingBox(BBFormat.XYX2Y2))
      print(iou)
      if(iou>iou_max):
        iou_max = iou
    ious.append(iou_max)

In [0]:
ious

In [0]:
from utils import *

In [0]:
evaluator = Evaluator.Evaluator()

In [0]:
class BoundingBox:
    def __init__(self,
                 imageName,
                 classId,
                 x,
                 y,
                 w,
                 h,
                 typeCoordinates=CoordinatesType.Absolute,
                 imgSize=None,
                 bbType=BBType.GroundTruth,
                 classConfidence=None,
                 format=BBFormat.XYWH):

In [0]:
evaluator.iou()

In [0]:
boxes = pd.read_csv('test.csv')

In [0]:
!python Object_Detection_Metrics/pascalvoc.py -gt ../ground_truth/ -det ../detections/ -gtformat xyrb -detformat xyrb -imgsize 1536,512 

In [0]:
evaluator = Evaluator.Evaluator()

In [0]:
evaluator.GetPascalVOCMetrics(boundingboxes, IOUThreshold=0.5,
                            method=MethodAveragePrecision.EveryPointInterpolation)

In [0]:
boxes_train = pd.read_csv('train.csv')

In [0]:
def _get_dicts_test(folder='EndoCV_Det/HOLDOUT/images/'):
    path = folder
    def get_dicts_test():
        dataset_dicts = []
        img_list = os.listdir(path)
#         print(img_list)
        for idx, i in enumerate(img_list):
            record = {}
            try:
                img = plt.imread(path+i)
                height, width, _ = img.shape
            except:
                print(i)
                continue
            record["file_name"] = path+i
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            record["thing_classes"] = ["specularity","saturation",
                                          "artifact", "blur", "contrast", "bubbles",
                                          "instrument", "blood"]
            dataset_dicts.append(record)
        return dataset_dicts
    return get_dicts_test

In [0]:
folder = 'EndoCV_Det/TEST/Detection/'
cfg.confidence_threshold = 0.5
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
visualize_preds = True
n_samples = 10
trainer.resume_or_load(resume=False)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = cfg.confidence_threshold
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = cfg.confidence_threshold 

from detectron2.utils.visualizer import ColorMode

cfg.DATASETS.TEST = ("endo1_val", )
predictor = DefaultPredictor(cfg)

dataset_dicts = _get_dicts_test(folder=folder)()

for d in random.sample(dataset_dicts, n_samples):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get(f"endo1_val"), 
                   scale=0.8, 
#                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#         v = v.draw_instance_predictions(outputs["proposals"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])
#     os.makedirs(os.path.join(cfg.OUTPUT_DIR, f'{t}_preds/'), exist_ok=True)

    # WandB
#     wandb.log({f"{t}: {d['file_name']}": [wandb.Image(v.get_image(), caption=f"Threshold: {cfg.confidence_threshold}")]})

In [0]:
ls EndoCV_Det/TEST/


In [0]:
outputs['proposals'].get_fields().keys()

In [0]:
nms_thresh = 0.1
score_thresh = 0.7

for d in random.sample(dataset_dicts, 2):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get(f"endo1_{t}"), 
                   scale=0.8, 
#                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )


    boxes = outputs['proposals'].get_fields()['proposal_boxes'].tensor.to('cpu')
    scores = outputs['proposals'].get_fields()['objectness_logits'].sigmoid().to('cpu')

    nms_idx = torchvision.ops.nms(boxes, scores, nms_thresh)
    score_idx = torch.arange(0, len(boxes))[scores >= score_thresh]
    idx = torch.tensor(list(set(list(score_idx.cpu().numpy())).intersection(set(list(nms_idx.cpu().numpy())))))

    v=v.overlay_instances(boxes=boxes[idx.cuda()])
    cv2_imshow(v.get_image()[:, :, ::-1])

In [0]:
from detectron2.utils.visualizer import ColorMode

cfg.DATASETS.TEST = ("endo1_val", )
predictor = DefaultPredictor(cfg)

for t in ['train', 'val']:
    dataset_dicts = _get_dicts(t)()

    print("\n"*2 + t.upper() + "\n"*2)
    for d in random.sample(dataset_dicts, 2):
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=MetadataCatalog.get(f"endo1_{t}"), 
                       scale=0.8, 
#                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
        )
        
#         v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        b=outputs['proposals'].get_fields()['proposal_boxes'].tensor.to('cpu')[outputs['proposals'].get_fields()['objectness_logits'].sigmoid() >= 0.8]
        v=v.overlay_instances(boxes=b) #
        cv2_imshow(v.get_image()[:, :, ::-1])
        os.makedirs(os.path.join(cfg.OUTPUT_DIR, f'{t}_preds/'), exist_ok=True)
        
        # WandB
        wandb.log({f"{t}: {d['file_name']}": [wandb.Image(v.get_image(), caption=f"Threshold: {cfg.confidence_threshold}")]})

In [0]:
??Visualizer

In [0]:
# from detectron2.data import DatasetCatalog, MetadataCatalog
# DatasetCatalog.register("endo1_test", _get_dicts_test(folder='EndoCV_Det/TEST/Detection/'))

# MetadataCatalog.get("endo1_test").set(thing_classes=["specularity","saturation",
#                                   "artifact", "blur", "contrast", "bubbles",
#                                   "instrument", "blood"])

# endo_metadata = MetadataCatalog.get("endo1_test") 

In [0]:
folder = 'EndoCV_Det/TRAIN/images/'


In [0]:
len(os.listdir('EndoCV2020_testSubmission/detection_bbox'))

In [0]:
# EndoCV2020_testSubmission/detection_bbox

In [0]:
!mkdir EndoCV2020_testSubmission/generalization_bbox

In [0]:
from tqdm import tqdm_notebook as tqdm

In [0]:
for i in tqdm(range(len(outputs))):
    output = outputs[i]['instances'].get_fields()
    create_txt(output, 'ead2020_testG_'+str(i+1).zfill(4))

# Other types of builtin models

In [0]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("endo1_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "endo1_vall")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

In [0]:
# Inference with a keypoint detection model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [0]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(v.get_image()[:, :, ::-1])

# Run panoptic segmentation on a video

In [0]:
# This is the video we're going to process
from IPython.display import YouTubeVideo, display
video = YouTubeVideo("ll8TgCZ0plk", width=500)
display(video)

In [0]:
# Install dependencies, download the video, and crop 5 seconds for processing
!pip install youtube-dl
!pip uninstall -y opencv-python opencv-contrib-python
!apt install python3-opencv  # the one pre-installed have some issues
!youtube-dl https://www.youtube.com/watch?v=ll8TgCZ0plk -f 22 -o video.mp4
!ffmpeg -i video.mp4 -t 00:00:06 -c:v copy video-clip.mp4

In [0]:
# Run frame-by-frame inference demo on this video (takes 3-4 minutes)
# Using a model trained on COCO dataset
!cd detectron2_repo && python demo/demo.py --config-file configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml --video-input ../video-clip.mp4 --confidence-threshold 0.6 --output ../video-output.mkv \
  --opts MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl

In [0]:
# Download the results
from google.colab import files
files.download('video-output.mkv')

In [0]:
del cfg
del trainer
import torch
torch.cuda.empty_cache()

In [0]:
def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [0]:
dump_tensors()

In [0]:
from detectron2.utils.visualizer import ColorMode

cfg.DATASETS.TEST = ("endo1_val", )
predictor = DefaultPredictor(cfg)

for t in ['train', 'val']:
    dataset_dicts = _get_dicts(t)()

    print("\n"*2 + t.upper() + "\n"*2)
    for d in random.sample(dataset_dicts, 10):
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                       metadata=MetadataCatalog.get(f"endo1_{t}"), 
                       scale=0.8, 
#                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#         v = v.draw_instance_predictions(outputs["proposals"].to("cpu"))
        cv2_imshow(v.get_image()[:, :, ::-1])
#         os.makedirs(os.path.join(cfg.OUTPUT_DIR, f'{t}_preds/'), exist_ok=True)
        
        # WandB
#         wandb.log({f"{t}: {d['file_name']}": [wandb.Image(v.get_image(), caption=f"Threshold: {cfg.confidence_threshold}")]})